In [73]:
import random
import math
import itertools
import numpy as np
from sklearn.cluster import spectral_clustering, SpectralClustering
from sklearn.metrics.cluster import normalized_mutual_info_score
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
import datetime
import matplotlib.pyplot as plt
from sklearn import cluster, datasets, mixture

import warnings
warnings.filterwarnings('ignore')

n_samples = 6000

def accuracy(labels, true_labels):
    total_common = 0
    cluster_names = set(true_labels)
    permutations = list(itertools.permutations(cluster_names))
    for permutation in permutations:
        max_common = 0
        for i, cluster_name in enumerate(cluster_names):
            cluster_nodes = np.where(labels == cluster_name)[0]
            cluster_name1 = permutation[i]
            true_nodes = np.where(true_labels == cluster_name1)[0]

            common = len(set(true_nodes) - (set(true_nodes) - set(cluster_nodes)))
            max_common += common

        total_common = max(total_common, max_common)

    return total_common / len(true_labels)

def get_B_and_weight_vec_ring(points, threshhold=0.2):
    N = len(points)
    clustering = SpectralClustering(n_clusters=2, assign_labels="discretize", random_state=0).fit(points)
    A = clustering.affinity_matrix_

    row = []
    col = []
    data = []
    weight_vec = []
    cnt = 0
    for i in range(N):
        for j in range(N):
            if j <= i:
                continue
            if A[i, j] < threshhold:
                A[i, j] = 0
                A[j, i] = 0
                continue
            row.append(cnt)
            col.append(i)
            data.append(1)

            row.append(cnt)
            col.append(j)
            data.append(-1)
            cnt += 1
            weight_vec.append(A[i, j])

    B = csr_matrix((data, (row, col)), shape=(cnt, N))
    weight_vec = np.array(weight_vec)
    return A, B, weight_vec

     
def algorithm1(B, weight_vec, samplingset, K, alpha, lambda_nLasso):
    E, N = B.shape
    
    seednodesindicator = np.zeros(N)
    seednodesindicator[samplingset] = 1
    noseednodeindicator = np.ones(N)
    noseednodeindicator[samplingset] = 0

    Gamma_vec = np.array(1. / (np.sum(abs(B), 0)))[0]  # \in [0, 1]
    Gamma = np.diag(Gamma_vec)

    Sigma = 0.5

    fac_alpha = 1. / (Gamma_vec * alpha + 1)  # \in [0, 1]

    hatx = np.zeros(N)
    newx = np.zeros(N)
    prevx = np.zeros(N)
    haty = np.array([x / (E - 1) for x in range(0, E)])
    lambda_weight = lambda_nLasso * weight_vec
    gamma_plus = 1 + Gamma_vec[samplingset]
    start = datetime.datetime.now()
    for iterk in range(K):
        tildex = 2 * hatx - prevx
        newy = haty + Sigma * B.dot(tildex)  # chould be negative
        res = abs(newy) / lambda_weight
        res[res < 1] = 1
        haty = newy / res

        newx = hatx - Gamma_vec * B.T.dot(haty)  # could  be negative

        newx[samplingset] = (newx[samplingset] + Gamma_vec[samplingset]) / gamma_plus

        newx = seednodesindicator * newx + noseednodeindicator * (newx * fac_alpha)
        prevx = np.copy(hatx)
        hatx = newx  # could be negative
    
    return newx
    


In [74]:
noisy_circles = datasets.make_circles(n_samples=n_samples, factor=0.3, noise=.05) 

points = noisy_circles[0] * 2
true_labels = noisy_circles[1]

A, B, weight_vec = get_B_and_weight_vec_ring(points, threshhold=0.2)




In [84]:
M = 0.05
samplingset = random.choices([i for i in range(len(points))], k=int(M * len(points)))

len(samplingset)

300

In [86]:
K=50
alpha, lambda_nLasso = 0.1, 0.01


for alpha in [0.1, 0.01, 0.001]:
    for lambda_nLasso in [0.5, 0.1, 0.01, 0.001]:
        signals = []
        for node in samplingset:
            res = algorithm1(B, weight_vec, [node], K, alpha, lambda_nLasso)
            signals.append(res)
        signals = np.array(signals)
        mean_signals = np.mean(signals, axis=0)
        X = np.nan_to_num(mean_signals, 0)
        kmeans = KMeans(n_clusters=2, random_state=0).fit(X.reshape(len(X), 1))
        our_accuracy = accuracy(kmeans.labels_, true_labels)
        print('alpha', alpha, 'lambda_nLasso', lambda_nLasso, 'accuracy', our_accuracy)
        print('mean_signals', mean_signals[:20])
        print('kmeans.labels_', kmeans.labels_[:20])
        print('true_labels', true_labels[:20])
        print('-------------------------------------')




alpha 0.1 lambda_nLasso 0.5 accuracy 0.7881666666666667
mean_signals [ 1.35751354e-04 -2.12155887e-04  3.35195888e-05  3.11634078e-05
  3.22935809e-05  2.14325653e-05 -2.14159970e-04  1.02617223e-04
  1.04579002e-04 -8.45800083e-06 -2.47355027e-04 -1.84029250e-04
 -2.70698908e-04  2.25339488e-05 -5.77287574e-05  3.16198049e-05
  3.89202333e-05  3.13531044e-05  1.30700090e-04  3.04782712e-05]
kmeans.labels_ [0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0]
true_labels [0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1]
-------------------------------------
alpha 0.1 lambda_nLasso 0.1 accuracy 0.8023333333333333
mean_signals [ 1.78156107e-05 -4.09868190e-05  1.18878043e-05  1.16557981e-05
  1.17857550e-05  9.66835626e-06 -4.11527085e-05  1.30967806e-05
  1.17289717e-05 -6.17817484e-06 -4.95805335e-05 -3.54384398e-05
 -5.40575924e-05  9.85637848e-06 -1.08633170e-05  1.17246144e-05
  1.51273432e-06  1.16937776e-05  1.66612508e-05  1.15439596e-05]
kmeans.labels_ [0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0]
true

In [87]:
for alpha in [0.1, 0.01, 0.001]:
    for lambda_nLasso in [0.5, 0.1, 0.01, 0.001]:
        signals = []
        for node in samplingset:
            res = algorithm1(B, weight_vec, [node], K, alpha, lambda_nLasso)
            signals.append(res)
        signals = np.array(signals)
        signals = signals.T
        X = np.nan_to_num(signals, 0)
        kmeans = KMeans(n_clusters=2, random_state=0).fit(X)
        our_accuracy = accuracy(kmeans.labels_, true_labels)
        print('alpha', alpha, 'lambda_nLasso', lambda_nLasso, 'accuracy', our_accuracy)
        print('kmeans.labels_', kmeans.labels_[:20])
        print('true_labels', true_labels[:20])
        print('-------------------------------------')



alpha 0.1 lambda_nLasso 0.5 accuracy 0.7881666666666667
kmeans.labels_ [0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0]
true_labels [0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1]
-------------------------------------
alpha 0.1 lambda_nLasso 0.1 accuracy 0.8021666666666667
kmeans.labels_ [0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0]
true_labels [0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1]
-------------------------------------
alpha 0.1 lambda_nLasso 0.01 accuracy 0.8126666666666666
kmeans.labels_ [0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0]
true_labels [0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1]
-------------------------------------
alpha 0.1 lambda_nLasso 0.001 accuracy 0.5001666666666666
kmeans.labels_ [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
true_labels [0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1]
-------------------------------------
alpha 0.01 lambda_nLasso 0.5 accuracy 0.7881666666666667
kmeans.labels_ [0 1 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0]
true_labels [0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1]
---------